In [1]:
from dataset.tfrecord_util import recursive_parse_xml_to_dict
from dataset.create_pascal_tfrecord import dict_to_tf_example
from dataset.create_pascal_tfrecord import UniqueId
#from dataset.create_pascal_tfrecord import define_flags

In [2]:
from lxml import etree
import tensorflow as tf
import PIL.Image
#from absl import flags
import json

In [3]:
import os,glob

In [4]:
#FLAGS = flags.FLAGS
#define_flags()
num_shards = 100
ignore_difficult_instances = False

In [5]:
output_path = "_logs/"

In [6]:
dataset_path = '/host/04 - Plovila Detection/data/01 active - open images data and labels'

In [7]:
pascal_label_map_dict = {
    'Boat': 1,
    'Jet ski': 2
}

In [8]:
ann_json_dict = {
      'images': [],
      'type': 'instances',
      'annotations': [],
      'categories': []
}
unique_id = UniqueId()

In [9]:
for class_name, class_id in pascal_label_map_dict.items():
      cls = {'supercategory': 'none', 'id': 'Boat', 'name': class_name}
      ann_json_dict['categories'].append(cls)

In [10]:
from sklearn.model_selection import train_test_split as ttsplit
x = os.listdir(f"{dataset_path}/labels")
x_train, x_test= ttsplit(x, test_size=0.15,random_state=42)

In [11]:
print(len(x_train),len(x_test))

21648 3821


## TRAIN DATASET TO TFRECORD

In [12]:

writers = [
    tf.io.TFRecordWriter(output_path + 'train-%05d-of-%05d.tfrecord' %
                         (i, num_shards))
    for i in range(num_shards)
]

In [13]:
full_paths = [f"{dataset_path}/labels/{name}" for name in x_train]

In [14]:


for idx,xml_name in enumerate(full_paths):
  tree = etree.parse(xml_name)
  root = tree.getroot()
  xml_dict = recursive_parse_xml_to_dict(root)
  
  img_dir = xml_dict['annotation']['filename']
  img_dir = f"{dataset_path}/data/"
  data = xml_dict['annotation']
  if idx % 1000 == 0 :
    print(f"Done: {idx}")

  tf_example = dict_to_tf_example(
          data,
          img_dir,
          pascal_label_map_dict,
          unique_id,
          ignore_difficult_instances,
          ann_json_dict=ann_json_dict)
  writers[idx % num_shards].write(tf_example.SerializeToString())


for writer in writers:
    writer.close()

json_file_path = os.path.join(
    os.path.dirname(output_path),
    'json_' + os.path.basename(output_path) + '.json')
with tf.io.gfile.GFile(json_file_path, 'w') as f:
  json.dump(ann_json_dict, f)

Done: 0
Done: 1000
Done: 2000
Done: 3000
Done: 4000
Done: 5000
Done: 6000
Done: 7000
Done: 8000
Done: 9000
Done: 10000
Done: 11000
Done: 12000
Done: 13000
Done: 14000
Done: 15000
Done: 16000
Done: 17000
Done: 18000
Done: 19000
Done: 20000
Done: 21000


## VALIDATION DATASET TO TFRECORD

In [15]:
writers = [
    tf.io.TFRecordWriter(output_path + 'val-%05d-of-%05d.tfrecord' %
                         (i, num_shards))
    for i in range(num_shards)
]

In [16]:
full_paths = [f"{dataset_path}/labels/{name}" for name in x_test]

In [17]:
for idx,xml_name in enumerate(full_paths):
  tree = etree.parse(xml_name)
  root = tree.getroot()
  xml_dict = recursive_parse_xml_to_dict(root)
  
  img_dir = xml_dict['annotation']['filename']
  img_dir = f"{dataset_path}/data/"
  data = xml_dict['annotation']
  if idx % 1000 == 0 :
    print(f"Done: {idx}")

  tf_example = dict_to_tf_example(
          data,
          img_dir,
          pascal_label_map_dict,
          unique_id,
          ignore_difficult_instances,
          ann_json_dict=ann_json_dict)
  writers[idx % num_shards].write(tf_example.SerializeToString())


for writer in writers:
    writer.close()

json_file_path = os.path.join(
    os.path.dirname(output_path),
    'val_json_' + os.path.basename(output_path) + '.json')
with tf.io.gfile.GFile(json_file_path, 'w') as f:
  json.dump(ann_json_dict, f)

Done: 0
Done: 1000
Done: 2000
Done: 3000
